In [ ]:
import pandas as pd
import numpy as np
from shapely.wkt import loads
from shapely.geometry import Point
from scipy.spatial import distance

In [ ]:
# Load known points CSV
known_csv = "climada_output_01.csv"  # Change to actual file path
df_known = pd.read_csv(known_csv)

# Load target points CSV
target_csv = "assets.csv"  # Change to actual file path
df_target = pd.read_csv(target_csv)

In [ ]:
# Ensure required columns exist
if "Latitude" not in df_target.columns or "Longitude" not in df_target.columns:
    raise ValueError("Target CSV must contain 'Latitude' and 'Longitude' columns.")

# Convert latitude & longitude to Point geometry
df_target["geometry"] = df_target.apply(lambda row: Point(row["Longitude"], row["Latitude"]), axis=1)

# Save and reload target file (optional, avoids potential formatting issues)
df_target.to_csv("assets_coords.csv", index=False)
df_target = pd.read_csv("assets_coords.csv")

# Parse WKT geometry in known points
df_known["geometry"] = df_known["geometry"].apply(lambda x: loads(x).coords[0])  # Convert WKT to (x, y)

# Convert target points to (x, y) tuples
df_target["geometry"] = df_target.apply(lambda row: (row["Longitude"], row["Latitude"]), axis=1)

In [ ]:
# Function to find the nearest known point for a given target
def find_nearest(target_point):
    df_known["distance"] = df_known["geometry"].apply(lambda x: distance.euclidean(target_point, x))
    nearest_row = df_known.loc[df_known["distance"].idxmin()]
    return nearest_row.drop(["geometry", "distance"])  # Drop extra columns

# Apply function and **expand results into separate columns**
df_nearest = df_target["geometry"].apply(find_nearest).apply(pd.Series)

# Merge nearest values back into target DataFrame
df_result = df_target.drop(columns=["geometry"]).join(df_nearest)

df_result.columns = ["Name", "SBU", "Latitude", "Longitude", "1-min MSW 10 yr RP", "1-min MSW 20 yr RP", "1-min MSW 50 yr RP", "1-min MSW 100 yr RP"]

In [ ]:
# List the column names you want to round
columns_to_round = ["1-min MSW 10 yr RP", "1-min MSW 20 yr RP", "1-min MSW 50 yr RP", "1-min MSW 100 yr RP"]
# Apply "round half up" while preserving NaNs
df_result[columns_to_round] = df_result[columns_to_round].apply(
    lambda col: np.floor(col + 0.5) if col.dtype.kind in 'fc' else col
).astype("Int64")

In [ ]:
# Define percent changes for the 75th percentile future projections
percent_changes_75p = {
    "4.5": {"2025-2035": 0.0203, "2035-2045": 0.0279, "2045-2055": 0.0351},
    "8.5": {"2025-2035": 0.0237, "2035-2045": 0.0355, "2045-2055": 0.0493}
}

# Identify all 1-min MSW RP columns (not just 100 yr)
rp_columns = [col for col in df_result.columns if "1-min" in col and "yr RP" in col]
if not rp_columns:
    raise ValueError("No '1-min ... yr RP' columns found!")

# Initialize result dataframe with identifiers
result_df = df_result[["Name", "SBU", "Latitude", "Longitude"]].copy()

# Process each RP column
for rp_col in rp_columns:
    # Add base/current values
    rp_label = rp_col.replace("1-min MSW ", "")  # e.g. "100 yr RP"
    base_col_name = f"{rp_label}_Current"
    result_df[base_col_name] = df_result[rp_col]

    # Loop through each scenario/year and apply percent changes
    for scenario, years in percent_changes_75p.items():
        for year, pct in years.items():
            col_name = f"{rp_label}_RCP{scenario}_{year}"
            adjusted_values = df_result[rp_col] * (1 + pct)

            # Round half up while keeping NaNs
            adjusted_values = adjusted_values.apply(
                lambda x: np.floor(x + 0.5) if pd.notnull(x) else pd.NA
            )
            result_df[col_name] = adjusted_values.astype("Int64")

# Save the compiled current climate and future climate exposure data to a single CSV
result_df.to_csv("tc_exposure_results.csv", index=False)
print("Saved: Compiled current and 75th percentile future projections")

result_df